In [1]:
import numpy as np

from vlmc import VLMC, BICSolver, ContextAlgorithmSolver
from vlmc.utils import count_word_ocurrences

import re
import glob
import copy
glob.glob('./*')

['./bct_solver.py',
 './vlmc.py',
 './context_algorithm_solver.py',
 './__init__.py',
 './counter.py',
 './utils.py',
 './bic_solver.py',
 './vlmc']

In [2]:
def parse_txt_input(input_path):
    """
        Função para ler e decodificar arquivos txt. Lê-se o arquivo e analisa-se cada linha
        para obter o respectivo valor do estado. Retorna-se um array com a sequência de estados observados (observed_sequence).
    """

    observed_sequence = []
    with open(
        input_path,
        mode='r',
        encoding='utf-8'
    ) as f:
        lines = f.readlines()
        # Use regular expression to obtain sequence from input
        observed_sequence = [
            int(i) for i in re.search('[0-9]{1000,100000}', lines[-1]).group(0)
        ]
        
    return observed_sequence

# BIC

### 1st Order sample

In [3]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [0.2, 0.8],
    [0.5, 0.5]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    current_s = X[i]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[current_s,:]
        )
    )

In [4]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [5]:
chain.fit(
    X=sample,
    method='bic',
    njobs=1
)
chain.show_tree()

root
├── 0
└── 1



In [6]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.20218240115843095, '1': 0.797817598841569},
 '1': {'0': 0.5031391158292974, '1': 0.4969261125515712}}

In [7]:
Q[0], Q[1]

(array([0.2, 0.8]), array([0.5, 0.5]))

### 2nd Order sample

In [8]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [[0.2, 0.8],
    [0.1, 0.9]],
    [[0.4, 0.6],
    [0.8, 0.2]]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    # import pdb;pdb.set_trace()
    current_s = X[i-1:i+1]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[tuple(current_s)]
        )
    )

In [9]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 0, 1, 1, 1, 0, 1, 1, 0])

In [10]:
chain.fit(
    X=sample,
    method='bic',
    njobs=10
)
chain.show_tree()

root
├── 0
│   ├── 00
│   └── 10
└── 1
    ├── 01
    └── 11



In [11]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'00': {'0': 0.20077386070507308, '1': 0.7992261392949269},
 '10': {'0': 0.4055410122164049, '1': 0.5944953461314718},
 '01': {'0': 0.09984365341962695, '1': 0.9001927062502273},
 '11': {'0': 0.7978281184545484, '1': 0.20220410530725358}}

In [12]:
Q[0,0], Q[1,0], Q[0,1], Q[1, 1]

(array([0.2, 0.8]), array([0.4, 0.6]), array([0.1, 0.9]), array([0.8, 0.2]))

### 4th order complex tree

In [13]:
def dirichlet_gen_fn():
    is_valid=False
    while not is_valid:
        probs = np.random.dirichlet(
            alpha = [0.5, 0.5, 0.5]
        )
        if max(probs)<0.8 and min(probs)>0.2:
            is_valid=True
    return probs

In [14]:
N_SAMPLES=1000000
MAX_DEPTH = 4
STATE_SPACE = [
    0,
    1,
    2
]

contexts = [
    '0',
    '11',
    '21',
    '001',
    '101',
    '0201',
    '1201',
    '2201',
    '02',
    '12',
    '022',
    '122',
    '222'
]

context_probs = {
    c: dirichlet_gen_fn() for c in contexts
}
context_probs

{'0': array([0.31864081, 0.34611793, 0.33524125]),
 '11': array([0.47350405, 0.25550706, 0.27098889]),
 '21': array([0.20026899, 0.22089417, 0.57883685]),
 '001': array([0.4217682 , 0.32946914, 0.24876266]),
 '101': array([0.2047283 , 0.44162982, 0.35364188]),
 '0201': array([0.35185035, 0.3982676 , 0.24988205]),
 '1201': array([0.28817417, 0.40405546, 0.30777037]),
 '2201': array([0.31144029, 0.25643527, 0.43212444]),
 '02': array([0.21221545, 0.25673943, 0.53104512]),
 '12': array([0.45217178, 0.32291338, 0.22491484]),
 '022': array([0.31558255, 0.32329551, 0.36112194]),
 '122': array([0.51130114, 0.25630526, 0.2323936 ]),
 '222': array([0.29589254, 0.36638082, 0.33772664])}

In [15]:
def context_identification_fn(full_word, ctxts):
    context_found = False
    context=None
    word = copy.deepcopy(full_word)
    while not context_found:
        if len(word)==0:
            import pdb;pdb.set_trace()
        if word in ctxts:
            context=word
            context_found=True
        else:
            word = word[1:]
            
            
    return context

In [16]:
sample = [0, 0, 1, 1]
i=MAX_DEPTH
while len(sample) < N_SAMPLES:
    word = ''.join(
        [str(x) for x in sample[i-MAX_DEPTH:i]]
    )
    ctxt = context_identification_fn(
        full_word=word,
        ctxts=contexts
    )
    new_state = np.random.choice(
        a=STATE_SPACE,
        p=context_probs[ctxt]
    )
    sample.append(new_state)
    i+=1
    

In [17]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
len(sample), sample[:10]

(1000000, [0, 0, 1, 1, 2, 0, 0, 2, 2, 0])

In [18]:
chain.fit(
    X=sample,
    method='bic',
    njobs=10
)
chain.show_tree()

root
├── 0
├── 1
│   ├── 01
│   │   ├── 001
│   │   ├── 101
│   │   └── 201
│   │       ├── 0201
│   │       ├── 1201
│   │       └── 2201
│   ├── 11
│   └── 21
└── 2
    ├── 02
    ├── 12
    └── 22
        ├── 022
        ├── 122
        └── 222



In [19]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.3180207155733447,
  '1': 0.34653365516819945,
  '2': 0.3354517400470531},
 '001': {'0': 0.4228482524865255,
  '1': 0.32811024059565486,
  '2': 0.24906928932599878},
 '101': {'0': 0.20839480459371112,
  '1': 0.4375885909340739,
  '2': 0.354016604472215},
 '0201': {'0': 0.345224541429475,
  '1': 0.4054395951929159,
  '2': 0.2493358633776091},
 '1201': {'0': 0.2861643340391468,
  '1': 0.4031916541858183,
  '2': 0.31064401177503487},
 '2201': {'0': 0.30824510747513634,
  '1': 0.2544112929098492,
  '2': 0.43734359961501446},
 '11': {'0': 0.46957343199991003,
  '1': 0.25684049528222314,
  '2': 0.27359731890821987},
 '21': {'0': 0.20117872079508745,
  '1': 0.22117487714032086,
  '2': 0.5776464020645917},
 '02': {'0': 0.2108590113763674,
  '1': 0.2568198999899808,
  '2': 0.5323210886336518},
 '12': {'0': 0.4527715000850526,
  '1': 0.3230541177614153,
  '2': 0.2241743821535321},
 '022': {'0': 0.3147511250278049,
  '1': 0.32296425577057986,
  '2': 0.36228461920161525},
 '122': {'0'

In [20]:
context_probs

{'0': array([0.31864081, 0.34611793, 0.33524125]),
 '11': array([0.47350405, 0.25550706, 0.27098889]),
 '21': array([0.20026899, 0.22089417, 0.57883685]),
 '001': array([0.4217682 , 0.32946914, 0.24876266]),
 '101': array([0.2047283 , 0.44162982, 0.35364188]),
 '0201': array([0.35185035, 0.3982676 , 0.24988205]),
 '1201': array([0.28817417, 0.40405546, 0.30777037]),
 '2201': array([0.31144029, 0.25643527, 0.43212444]),
 '02': array([0.21221545, 0.25673943, 0.53104512]),
 '12': array([0.45217178, 0.32291338, 0.22491484]),
 '022': array([0.31558255, 0.32329551, 0.36112194]),
 '122': array([0.51130114, 0.25630526, 0.2323936 ]),
 '222': array([0.29589254, 0.36638082, 0.33772664])}

# Context Algorithm

### 1st Order sample

In [21]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [0.2, 0.8],
    [0.5, 0.5]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    current_s = X[i]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[current_s,:]
        )
    )

In [22]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 0, 1, 0, 1, 1, 1, 1, 0])

In [23]:
chain.fit(
    X=sample,
    method='context',
    njobs=1
)
chain.show_tree()

root
├── 0
└── 1



In [24]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.1985887621725772, '1': 0.8014372754257147},
 '1': {'0': 0.4997564539698003, '1': 0.5002922552362397}}

In [25]:
Q[0], Q[1]

(array([0.2, 0.8]), array([0.5, 0.5]))

### 2nd Order sample

In [26]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [[0.2, 0.8],
    [0.1, 0.9]],
    [[0.4, 0.6],
    [0.8, 0.2]]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    # import pdb;pdb.set_trace()
    current_s = X[i-1:i+1]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[tuple(current_s)]
        )
    )

In [27]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [28]:
chain.fit(
    X=sample,
    method='context',
    njobs=10
)
chain.show_tree()

root
├── 0
│   ├── 00
│   └── 10
└── 1
    ├── 01
    └── 11



In [29]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'00': {'0': 0.20469798657718122, '1': 0.7953020134228188},
 '10': {'0': 0.4056366747988202, '1': 0.5943997378290792},
 '01': {'0': 0.09798638167716564, '1': 0.9020136183228343},
 '11': {'0': 0.7974826165336081, '1': 0.2025817666752511}}

In [30]:
Q[0,0], Q[1,0], Q[0,1], Q[1, 1]

(array([0.2, 0.8]), array([0.4, 0.6]), array([0.1, 0.9]), array([0.8, 0.2]))

### 4th order complex tree

In [31]:
def dirichlet_gen_fn():
    is_valid=False
    while not is_valid:
        probs = np.random.dirichlet(
            alpha = [0.5, 0.5, 0.5]
        )
        if max(probs)<0.8 and min(probs)>0.2:
            is_valid=True
    return probs

In [32]:
N_SAMPLES=1000000
MAX_DEPTH = 4
STATE_SPACE = [
    0,
    1,
    2
]

contexts = [
    '0',
    '11',
    '21',
    '001',
    '101',
    '0201',
    '1201',
    '2201',
    '02',
    '12',
    '022',
    '122',
    '222'
]

context_probs = {
    c: dirichlet_gen_fn() for c in contexts
}
context_probs

{'0': array([0.23194884, 0.33482282, 0.43322834]),
 '11': array([0.25113573, 0.48671173, 0.26215255]),
 '21': array([0.40541961, 0.30127287, 0.29330751]),
 '001': array([0.43609657, 0.27046525, 0.29343818]),
 '101': array([0.35680587, 0.32242059, 0.32077355]),
 '0201': array([0.58897249, 0.20744677, 0.20358074]),
 '1201': array([0.32614799, 0.41510292, 0.25874908]),
 '2201': array([0.34191507, 0.4358539 , 0.22223103]),
 '02': array([0.438764  , 0.22268923, 0.33854677]),
 '12': array([0.32952598, 0.4385052 , 0.23196883]),
 '022': array([0.42112011, 0.29419569, 0.2846842 ]),
 '122': array([0.30532478, 0.20746017, 0.48721505]),
 '222': array([0.35353038, 0.35448591, 0.2919837 ])}

In [33]:
def context_identification_fn(full_word, ctxts):
    context_found = False
    context=None
    word = copy.deepcopy(full_word)
    while not context_found:
        if len(word)==0:
            import pdb;pdb.set_trace()
        if word in ctxts:
            context=word
            context_found=True
        else:
            word = word[1:]
            
            
    return context

In [34]:
sample = [0, 0, 1, 1]
i=MAX_DEPTH
while len(sample) < N_SAMPLES:
    word = ''.join(
        [str(x) for x in sample[i-MAX_DEPTH:i]]
    )
    ctxt = context_identification_fn(
        full_word=word,
        ctxts=contexts
    )
    new_state = np.random.choice(
        a=STATE_SPACE,
        p=context_probs[ctxt]
    )
    sample.append(new_state)
    i+=1
    

In [35]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
len(sample), sample[:10]

(1000000, [0, 0, 1, 1, 1, 1, 2, 1, 0, 1])

In [36]:
chain.fit(
    X=sample,
    method='context',
    njobs=10
)
chain.show_tree()

root
├── 0
├── 1
│   ├── 01
│   │   ├── 001
│   │   ├── 101
│   │   └── 201
│   │       ├── 0201
│   │       ├── 1201
│   │       └── 2201
│   ├── 11
│   └── 21
└── 2
    ├── 02
    ├── 12
    └── 22
        ├── 022
        ├── 122
        └── 222



In [37]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.2315731776414653,
  '1': 0.3354950360899662,
  '2': 0.43293794496588084},
 '001': {'0': 0.4334439965220141,
  '1': 0.2664216267488736,
  '2': 0.3001738992964983},
 '101': {'0': 0.35352519793892173,
  '1': 0.32363956264923965,
  '2': 0.3228352394118386},
 '0201': {'0': 0.5913857860592167,
  '1': 0.20535583630066825,
  '2': 0.20325837764011512},
 '1201': {'0': 0.32567646486276036,
  '1': 0.41794821880474986,
  '2': 0.2563753163324898},
 '2201': {'0': 0.34927790937571634,
  '1': 0.4311912585008023,
  '2': 0.21953083212348132},
 '11': {'0': 0.253236098450319,
  '1': 0.48686140065792083,
  '2': 0.2599104276485276},
 '21': {'0': 0.405844566030768,
  '1': 0.30100389702523833,
  '2': 0.29315153694399365},
 '02': {'0': 0.43668293099278777,
  '1': 0.22192981208301918,
  '2': 0.3413872569241931},
 '12': {'0': 0.3280969624034298,
  '1': 0.4404138953657043,
  '2': 0.23148914223086595},
 '022': {'0': 0.42115132195762234,
  '1': 0.2962060128758047,
  '2': 0.28264266516657294},
 '122': {

In [38]:
context_probs

{'0': array([0.23194884, 0.33482282, 0.43322834]),
 '11': array([0.25113573, 0.48671173, 0.26215255]),
 '21': array([0.40541961, 0.30127287, 0.29330751]),
 '001': array([0.43609657, 0.27046525, 0.29343818]),
 '101': array([0.35680587, 0.32242059, 0.32077355]),
 '0201': array([0.58897249, 0.20744677, 0.20358074]),
 '1201': array([0.32614799, 0.41510292, 0.25874908]),
 '2201': array([0.34191507, 0.4358539 , 0.22223103]),
 '02': array([0.438764  , 0.22268923, 0.33854677]),
 '12': array([0.32952598, 0.4385052 , 0.23196883]),
 '022': array([0.42112011, 0.29419569, 0.2846842 ]),
 '122': array([0.30532478, 0.20746017, 0.48721505]),
 '222': array([0.35353038, 0.35448591, 0.2919837 ])}

# BCT

### 1st Order sample

In [39]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [0.2, 0.8],
    [0.5, 0.5]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    current_s = X[i]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[current_s,:]
        )
    )

In [40]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 1, 0, 1, 1, 0, 1, 0, 1])

In [41]:
chain.fit(
    X=sample,
    method='bct',
    njobs=1
)
chain.show_tree()

/home/leonardo/git/markov_chains/vlmc/vlmc/bct_solver.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prod_vectors = np.array([
/home/leonardo/git/markov_chains/vlmc/vlmc/bct_solver.py:90: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  prod_vectors = np.array([


root
├── 0
└── 1



In [42]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.1978131212723658, '1': 0.8022130375640891},
 '1': {'0': 0.4964868540344515, '1': 0.5035617148037819}}

In [43]:
Q[0], Q[1]

(array([0.2, 0.8]), array([0.5, 0.5]))

### 2nd Order sample

In [44]:
N_SAMPLES = 100000

# Declare state space and mapping from state number to zero-indexed position in transition matrix
STATE_SPACE = [
    0,
    1
]
STATE_TO_INDEX_MAP = {
    s: i for i, s in enumerate(STATE_SPACE)
}

Q = np.array([
    [[0.2, 0.8],
    [0.1, 0.9]],
    [[0.4, 0.6],
    [0.8, 0.2]]
])

X = [1, 1]
for i in range(len(X)-1, N_SAMPLES):
    # import pdb;pdb.set_trace()
    current_s = X[i-1:i+1]
    X.append(
        np.random.choice(
            a=STATE_SPACE,
            p=Q[tuple(current_s)]
        )
    )

In [45]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
sample=copy.deepcopy(X)
len(sample), sample[:10]

(100001, [1, 1, 1, 0, 0, 1, 1, 1, 0, 1])

In [46]:
chain.fit(
    X=sample,
    method='bct',
    njobs=10
)
chain.show_tree()

root
├── 0
│   ├── 00
│   └── 10
└── 1
    ├── 01
    └── 11



In [47]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'00': {'0': 0.2013933778639661, '1': 0.798606622136034},
 '10': {'0': 0.40349794985304255, '1': 0.5965383359338148},
 '01': {'0': 0.10003991436554302, '1': 0.899960085634457},
 '11': {'0': 0.8012598933936359, '1': 0.19880471652398643}}

In [48]:
Q[0,0], Q[1,0], Q[0,1], Q[1, 1]

(array([0.2, 0.8]), array([0.4, 0.6]), array([0.1, 0.9]), array([0.8, 0.2]))

### 4th order complex tree

In [49]:
def dirichlet_gen_fn():
    is_valid=False
    while not is_valid:
        probs = np.random.dirichlet(
            alpha = [0.5, 0.5, 0.5]
        )
        if max(probs)<0.8 and min(probs)>0.2:
            is_valid=True
    return probs

In [50]:
N_SAMPLES=1000000
MAX_DEPTH = 4
STATE_SPACE = [
    0,
    1,
    2
]

contexts = [
    '0',
    '11',
    '21',
    '001',
    '101',
    '0201',
    '1201',
    '2201',
    '02',
    '12',
    '022',
    '122',
    '222'
]

context_probs = {
    c: dirichlet_gen_fn() for c in contexts
}
context_probs

{'0': array([0.40196488, 0.21255887, 0.38547624]),
 '11': array([0.23142233, 0.44272205, 0.32585562]),
 '21': array([0.5458652 , 0.21410676, 0.24002804]),
 '001': array([0.5151316 , 0.20480388, 0.28006452]),
 '101': array([0.35265962, 0.33593962, 0.31140076]),
 '0201': array([0.21022434, 0.55621249, 0.23356317]),
 '1201': array([0.46889058, 0.32945988, 0.20164953]),
 '2201': array([0.57022559, 0.21741853, 0.21235588]),
 '02': array([0.31454732, 0.44456083, 0.24089185]),
 '12': array([0.5269841 , 0.25366146, 0.21935443]),
 '022': array([0.30134123, 0.44976172, 0.24889705]),
 '122': array([0.21064779, 0.36020874, 0.42914347]),
 '222': array([0.3100604 , 0.21947724, 0.47046236])}

In [51]:
def context_identification_fn(full_word, ctxts):
    context_found = False
    context=None
    word = copy.deepcopy(full_word)
    while not context_found:
        if len(word)==0:
            import pdb;pdb.set_trace()
        if word in ctxts:
            context=word
            context_found=True
        else:
            word = word[1:]
            
            
    return context

In [52]:
sample = [0, 0, 1, 1]
i=MAX_DEPTH
while len(sample) < N_SAMPLES:
    word = ''.join(
        [str(x) for x in sample[i-MAX_DEPTH:i]]
    )
    ctxt = context_identification_fn(
        full_word=word,
        ctxts=contexts
    )
    new_state = np.random.choice(
        a=STATE_SPACE,
        p=context_probs[ctxt]
    )
    sample.append(new_state)
    i+=1
    

In [53]:
chain = VLMC(
    vocabulary=STATE_SPACE,
    max_order=5
)
len(sample), sample[:10]

(1000000, [0, 0, 1, 1, 0, 2, 1, 0, 2, 0])

In [54]:
chain.fit(
    X=sample,
    method='bct',
    njobs=10
)
chain.show_tree()

root
├── 0
├── 1
│   ├── 01
│   │   ├── 001
│   │   ├── 101
│   │   └── 201
│   │       ├── 0201
│   │       ├── 1201
│   │       └── 2201
│   ├── 11
│   └── 21
└── 2
    ├── 02
    ├── 12
    └── 22
        ├── 022
        ├── 122
        └── 222



In [55]:
leaves = chain.get_leaves()
{n.word: n.transition_probabilities for n in leaves}

{'0': {'0': 0.4016368356890369,
  '1': 0.21271212809042636,
  '2': 0.38565613698616164},
 '001': {'0': 0.5113118098895824,
  '1': 0.20772323571771484,
  '2': 0.28099495919347095},
 '101': {'0': 0.3541465300599265,
  '1': 0.3381016818093949,
  '2': 0.30775178813067855},
 '0201': {'0': 0.21828789860175032,
  '1': 0.5551755356603963,
  '2': 0.22653656573785333},
 '1201': {'0': 0.47265884436572947,
  '1': 0.3263227806065973,
  '2': 0.20101837502767322},
 '2201': {'0': 0.5759312320916905,
  '1': 0.21298949379178606,
  '2': 0.2110792741165234},
 '11': {'0': 0.23235967990082682,
  '1': 0.442914540604558,
  '2': 0.32473684793075586},
 '21': {'0': 0.5461686738767015,
  '1': 0.21392992391934768,
  '2': 0.23990140220395073},
 '02': {'0': 0.31214495916410406,
  '1': 0.4453394173858414,
  '2': 0.24251562345005456},
 '12': {'0': 0.5292947988444656,
  '1': 0.25281692782912096,
  '2': 0.21788827332641347},
 '022': {'0': 0.29834751308900526,
  '1': 0.4511343804537522,
  '2': 0.2505181064572426},
 '122'

In [56]:
context_probs

{'0': array([0.40196488, 0.21255887, 0.38547624]),
 '11': array([0.23142233, 0.44272205, 0.32585562]),
 '21': array([0.5458652 , 0.21410676, 0.24002804]),
 '001': array([0.5151316 , 0.20480388, 0.28006452]),
 '101': array([0.35265962, 0.33593962, 0.31140076]),
 '0201': array([0.21022434, 0.55621249, 0.23356317]),
 '1201': array([0.46889058, 0.32945988, 0.20164953]),
 '2201': array([0.57022559, 0.21741853, 0.21235588]),
 '02': array([0.31454732, 0.44456083, 0.24089185]),
 '12': array([0.5269841 , 0.25366146, 0.21935443]),
 '022': array([0.30134123, 0.44976172, 0.24889705]),
 '122': array([0.21064779, 0.36020874, 0.42914347]),
 '222': array([0.3100604 , 0.21947724, 0.47046236])}